In [1]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

model_name = "deepset/roberta-base-squad2"

# a) Get predictions
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)
QA_input = {
    'question': 'Why is model conversion important?',
    'context': 'The option to convert models between FARM and transformers gives freedom to the user and let people easily switch between frameworks.'
}
res = nlp(QA_input)

# b) Load model & tokenizer
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/home/ashmitchamoli/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-09-15 12:45:09.586880: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-15 12:45:11.467389: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/ashmitchamoli/.local/lib/python3.11/site-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/home/ashmitchamoli/.local/lib

In [2]:
print(res)

{'score': 0.21171486377716064, 'start': 59, 'end': 84, 'answer': 'gives freedom to the user'}


In [14]:
import pandas as pd
from typing import Dict

trainDf = pd.read_csv('./dataset/train.csv')
imageMap : Dict[str, int] = { image : id for id, image in enumerate(trainDf['image_link'].unique()) }
trainDf['image_id'] = trainDf['image_link'].map(lambda x : imageMap.get(x, -1))

In [30]:
from alive_progress import alive_bar as aliveBar
from src.image import Image


with aliveBar(len(trainDf), force_tty=True) as bar:
	accuracy = 0
	trueAccuracy = 0
	for index, row in trainDf.iterrows():
		image = Image(row['image_link'], row['image_id'])
		image.getImage()

		question = f"What is the item {(row['entity_name'].split('_'))[-1]}?"
		context = image.readTextFrom()
		if context == "":
			context = "No context available."

		res = nlp(question=question, context=context)
		print(f"QUESTION: {question}\nANSWER: {res['answer']}\nCORRECT ANSWER: {row['entity_value']}\n")

		# print(res, context)
		value = res['answer'].split()
		if(len(value) == 0):
			value = " "
		else:
			value = value[0]
		accuracy += (1 if value == row['entity_value'].split()[0] else 0)
		trueAccuracy += (res['answer'] == row['entity_value'])

		bar.text(f"Accuracy: {accuracy}/{index + 1} = {(accuracy / (index + 1) * 100):.3f}%\nTrue Acc.: {trueAccuracy}/{index + 1} = {(trueAccuracy / (index + 1) * 100):.3f}%")
		bar()

|                                        | ▁▃▅ 0/263859 [0%] in 0s (~0s, 0.0/s) 

on 0: QUESTION: What is the item weight?                                        
      ANSWER: 100% NATUR
      
      
      CORRECT ANSWER: 500.0 gram
on 1: QUESTION: What is the item volume?                                        
      ANSWER: 
      
      CORRECT ANSWER: 1.0 cup
on 2: QUESTION: What is the item weight?                                        
      ANSWER: 1 Tablet
      CORRECT ANSWER: 0.709 gram
on 3: QUESTION: What is the item weight?                                        
      ANSWER: No context available.
      CORRECT ANSWER: 0.709 gram
on 4: QUESTION: What is the item weight?                                        
      ANSWER: 
      PSYLLIUM
      
      HUSK
      
      
      CORRECT ANSWER: 1400 milligram
on 5: QUESTION: What is the item weight?                                        
      ANSWER: 1400mg
      CORRECT ANSWER: 1400 milligram
on 6: QUESTION: What is the item weight?                                        
      ANSWER: 
      HIGH S

KeyboardInterrupt: 